[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hw-oh/wandb_e2e_demo/blob/main/models/automations/automations.ipynb)

In [ ]:
!pip install -q wandb requests

In [ ]:
import wandb
import os
from google.colab import userdata

# === Colab Secrets에서 환경변수 로드 ===
# Colab 좌측 🔑 아이콘 → 아래 키들을 등록하세요
WANDB_API_KEY = userdata.get("WANDB_API_KEY")
WANDB_ENTITY = userdata.get("WANDB_ENTITY")       # W&B 사용자명 또는 팀명
WANDB_PROJECT = userdata.get("WANDB_PROJECT")       # 예: "wandb-e2e-demo-image-classification"
GITHUB_PAT = userdata.get("GITHUB_PAT")             # GitHub Personal Access Token (repo scope)
GITHUB_REPO = userdata.get("GITHUB_REPO")            # 예: "owner/wandb-e2e-demo"

os.environ["WANDB_API_KEY"] = WANDB_API_KEY

wandb.login(key=WANDB_API_KEY)
print(f"Entity: {WANDB_ENTITY}")
print(f"Project: {WANDB_PROJECT}")
print(f"GitHub Repo: {GITHUB_REPO}")

# W&B Automations — 자동 배포 파이프라인

## 전체 아키텍처

```
┌─────────────┐     ┌──────────────────┐     ┌─────────────────────┐     ┌──────────────────┐
│  Notebook   │     │  W&B Model       │     │  GitHub Actions     │     │  Streamlit       │
│  (Colab)    │───▶│  Registry        │───▶│  Workflow           │───▶│  Cloud           │
│             │     │                  │     │                     │     │                  │
│ 모델 승격    │     │ "production"     │     │ 모델 다운로드       │     │ 새 모델로 서빙   │
│ 실행        │     │ alias 승격 시    │     │ → 앱 재배포         │     │ → 추론 테스트    │
└─────────────┘     └──────────────────┘     └─────────────────────┘     └──────────────────┘
```

### 흐름
1. 학습 노트북에서 모델을 Model Registry에 "staging"으로 등록
2. **이 노트북**에서 모델을 "production"으로 승격
3. W&B Automation이 Webhook을 발동 → GitHub Actions `repository_dispatch`
4. GitHub Actions가 모델 Artifact 다운로드 + `deployment.json` 업데이트 + commit & push
5. Streamlit Cloud가 자동 재배포 → 새 모델로 서빙

## 사전 준비 사항

### 1. GitHub Personal Access Token (PAT)
- GitHub Settings → Developer settings → Personal access tokens → Tokens (classic)
- `repo` scope 필요
- Colab에서 `userdata`로 저장: 키 이름 `GITHUB_PAT`

### 2. GitHub Repository Secrets
- Repository Settings → Secrets and variables → Actions
- `WANDB_API_KEY`: W&B API 키 추가

### 3. W&B Webhook 설정
- W&B Settings → Webhooks → New Webhook
- **URL**: `https://api.github.com/repos/{owner}/{repo}/dispatches`
- **Auth**: `Bearer {GITHUB_PAT}` (repo scope)
- **Payload template**:
```json
{
  "event_type": "wandb-model-promoted",
  "client_payload": {
    "model_name": "${event.artifact_collection_name}",
    "model_version": "${event.artifact_version}",
    "artifact_path": "${event.artifact_version_string}",
    "event_author": "${event.event_author}"
  }
}
```

### 4. W&B Automation Rule
- W&B → Automations → New Automation
- **이벤트**: "An alias is added to an artifact version in a registered model"
- **필터**: alias = `production`
- **액션**: 위에서 만든 Webhook 선택

### 5. Streamlit Cloud
- [share.streamlit.io](https://share.streamlit.io) 에서 GitHub repo 연동
- Main file path: `models/app/app.py`
- Secrets에 `WANDB_API_KEY` 추가

In [ ]:
# === 설정 ===

REGISTERED_MODEL = "cifar10-classifier"  # Model Registry에 등록된 모델명

api = wandb.Api()
print(f"Entity: {WANDB_ENTITY}, Project: {WANDB_PROJECT}")
print(f"Registered Model: {REGISTERED_MODEL}")

In [ ]:
# === Registry 현황 조회 ===

collections = api.artifact_type("model", project=f"{WANDB_ENTITY}/{WANDB_PROJECT}").collections()

for collection in collections:
    print(f"\nModel: {collection.name}")
    print("-" * 50)
    for version in collection.versions():
        aliases = ", ".join(version.aliases) if version.aliases else "-"
        print(f"  {version.version} | aliases: [{aliases}] | created: {version.created_at}")

In [ ]:
# === Staging 모델 성능 비교 ===

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    job_type="model-evaluation",
    name="staging-model-comparison",
)

# Registry의 모든 버전 메트릭 수집
collection = api.artifact_collection("model", f"{WANDB_ENTITY}/{WANDB_PROJECT}/{REGISTERED_MODEL}")

comparison_data = []
for version in collection.versions():
    metadata = version.metadata
    source_run = version.logged_by()
    val_acc = metadata.get("best_val_acc", None)
    if val_acc is None and source_run:
        val_acc = source_run.summary.get("best_val_acc", "N/A")

    comparison_data.append([
        version.version,
        ", ".join(version.aliases) if version.aliases else "-",
        val_acc,
        str(version.created_at),
    ])

table = wandb.Table(
    columns=["Version", "Aliases", "Val Accuracy", "Created"],
    data=comparison_data,
)
wandb.log({"model_comparison": table})

print("Staging 모델 비교 테이블:")
for row in comparison_data:
    print(f"  {row[0]} | aliases: [{row[1]}] | acc: {row[2]} | created: {row[3]}")

wandb.finish()

In [ ]:
# === "production" 승격 실행 ===
# 승격할 버전을 선택하세요 (예: "latest" 또는 특정 버전 "v0")

PROMOTE_VERSION = "latest"  # 또는 "v0", "v1" 등 특정 버전

artifact = api.artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/{REGISTERED_MODEL}:{PROMOTE_VERSION}"
)

# 기존 production alias 제거 (있는 경우)
if "production" in artifact.aliases:
    print(f"{artifact.version}은 이미 production입니다.")
else:
    artifact.aliases.append("production")
    artifact.save()
    print(f"{artifact.name}:{artifact.version} → production 승격 완료!")
    print("\nW&B Automation이 GitHub Actions Webhook을 트리거합니다...")
    print("배포 파이프라인이 자동으로 실행됩니다.")

In [ ]:
# === GitHub Actions 실행 상태 확인 ===

import requests
import time

print("GitHub Actions 워크플로우 실행 대기 중... (30초)")
time.sleep(30)  # Webhook 전파 + 워크플로우 시작 대기

resp = requests.get(
    f"https://api.github.com/repos/{GITHUB_REPO}/actions/runs",
    headers={"Authorization": f"Bearer {GITHUB_PAT}"},
    params={"event": "repository_dispatch", "per_page": 5},
)

if resp.status_code == 200:
    runs = resp.json().get("workflow_runs", [])
    if runs:
        print("\n최근 배포 워크플로우:")
        for r in runs[:3]:
            status_emoji = {
                "completed": "done",
                "in_progress": "running",
                "queued": "queued",
            }.get(r["status"], r["status"])
            print(f"  [{status_emoji}] {r['name']} - {r['created_at']}")
            print(f"    URL: {r['html_url']}")
    else:
        print("repository_dispatch 워크플로우가 아직 없습니다.")
else:
    print(f"GitHub API 오류: {resp.status_code}")
    print(resp.json())

In [ ]:
# === 배포 상태 확인 ===

import json as _json

# deployment.json이 업데이트되었는지 GitHub API로 확인
resp = requests.get(
    f"https://api.github.com/repos/{GITHUB_REPO}/contents/models/app/deployment.json",
    headers={"Authorization": f"Bearer {GITHUB_PAT}"},
)

if resp.status_code == 200:
    import base64
    content = base64.b64decode(resp.json()["content"]).decode("utf-8")
    deploy_info = _json.loads(content)
    print("\n현재 배포 상태:")
    print(f"  모델: {deploy_info['model_name']}")
    print(f"  버전: {deploy_info['model_version']}")
    print(f"  배포 시각: {deploy_info['deployed_at']}")
else:
    print(f"deployment.json 조회 실패: {resp.status_code}")

In [ ]:
# === 배포 이벤트 로깅 ===

deploy_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    job_type="deployment",
    name=f"deploy-{artifact.version}",
    config={
        "model_name": REGISTERED_MODEL,
        "model_version": artifact.version,
        "artifact_path": f"{WANDB_ENTITY}/{WANDB_PROJECT}/{artifact.name}:{artifact.version}",
        "action": "production-promotion",
    },
)

wandb.log({
    "deployment_event": wandb.Table(
        columns=["Model", "Version", "Action", "Timestamp"],
        data=[[REGISTERED_MODEL, artifact.version, "production-promotion", str(time.strftime('%Y-%m-%dT%H:%M:%SZ'))]],
    )
})

wandb.finish()
print("배포 이벤트 로깅 완료!")

## 롤백 가이드

배포된 모델에 문제가 발생하면, 이전 버전을 다시 "production"으로 승격하여 롤백할 수 있습니다.

### 롤백 절차
1. 현재 production 모델의 문제 확인
2. 이전 안정 버전 식별 (Registry의 버전 이력 참조)
3. 이전 버전에 "production" alias 이동
4. 자동으로 배포 파이프라인 재실행

> **참고**: W&B Model Registry는 alias 이동 시에도 Automation을 트리거합니다.
> 롤백도 승격과 동일한 파이프라인을 타므로 별도 조치가 필요 없습니다.

In [ ]:
# === 롤백 실행 예시 ===
# 이전 버전으로 롤백하려면 아래 코드의 주석을 해제하고 버전을 수정하세요

# ROLLBACK_VERSION = "v0"  # 롤백할 버전
#
# rollback_artifact = api.artifact(
#     f"{WANDB_ENTITY}/{WANDB_PROJECT}/{REGISTERED_MODEL}:{ROLLBACK_VERSION}"
# )
#
# # 현재 production에서 alias 제거
# current_prod = api.artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/{REGISTERED_MODEL}:production")
# current_prod.aliases.remove("production")
# current_prod.save()
#
# # 이전 버전에 production alias 추가
# rollback_artifact.aliases.append("production")
# rollback_artifact.save()
# print(f"롤백 완료! {ROLLBACK_VERSION} → production")

print("롤백 예시 코드입니다. 필요 시 주석을 해제하여 실행하세요.")